# Mimikatz Detection LSASS Access
Detects process access to LSASS which is typical for Mimikatz (0x1000 PROCESS_QUERY_ LIMITED_INFORMATION, 0x0400 PROCESS_QUERY_ INFORMATION "only old versions", 0x0010 PROCESS_VM_READ)

## Rule Content
```
- title: Mimikatz Detection LSASS Access
  id: 0d894093-71bc-43c3-8c4d-ecfc28dcf5d9
  status: experimental
  description: Detects process access to LSASS which is typical for Mimikatz (0x1000
    PROCESS_QUERY_ LIMITED_INFORMATION, 0x0400 PROCESS_QUERY_ INFORMATION "only old
    versions", 0x0010 PROCESS_VM_READ)
  references:
  - https://onedrive.live.com/view.aspx?resid=D026B4699190F1E6!2843&ithint=file%2cpptx&app=PowerPoint&authkey=!AMvCRTKB_V1J5ow
  - https://cyberwardog.blogspot.com/2017/03/chronicles-of-threat-hunter-hunting-for_22.html
  tags:
  - attack.t1003
  - attack.s0002
  - attack.credential_access
  - car.2019-04-004
  logsource:
    product: windows
    service: sysmon
    category: null
  detection:
    selection:
      EventID: 10
      TargetImage: C:\windows\system32\lsass.exe
      GrantedAccess:
      - '0x1410'
      - '0x1010'
    condition: selection
  falsepositives:
  - unknown
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-sysmon-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"10" AND target_process_path:"C\:\\windows\\system32\\lsass.exe" AND process_granted_access:("0x1410" OR "0x1010"))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()